# Multi-Criteria Decision Analysis of Top Twitch Streamers


In [22]:
import pandas as pd
import numpy as np
import time

### I- Data Cleaning

In [23]:
df = pd.read_csv("twitchdata.csv")

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Channel               1000 non-null   object
 1   Watch time(Minutes)   1000 non-null   int64 
 2   Stream time(minutes)  1000 non-null   int64 
 3   Peak viewers          1000 non-null   int64 
 4   Average viewers       1000 non-null   int64 
 5   Followers             1000 non-null   int64 
 6   Followers gained      1000 non-null   int64 
 7   Views gained          1000 non-null   int64 
 8   Partnered             1000 non-null   bool  
 9   Mature                1000 non-null   bool  
 10  Language              1000 non-null   object
dtypes: bool(2), int64(7), object(2)
memory usage: 72.4+ KB


#### Filtering Mature Streamers

To focus on streamers with mature content, we filter the dataset where the **Mature** column is `True`.  
This helps in analyzing content that may have different audience engagement patterns compared to general streaming channels.  

##### Selecting the Top 200 Streamers by Followers

After filtering for mature content, we select the **top 200 streamers** based on the number of **Followers**.  
This ensures that we analyze the most influential mature streamers with the largest audience reach.  


In [25]:
df.head()

,Channel,Watch time(Minutes),Stream time(minutes),Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language
0,xQcOW,6196161750,215250,222720,27716,3246298,1734810,93036735,True,False,English
1,summit1g,6091677300,211845,310998,25610,5310163,1370184,89705964,True,False,English
2,Gaules,5644590915,515280,387315,10976,1767635,1023779,102611607,True,True,Portuguese
3,ESL_CSGO,3970318140,517740,300575,7714,3944850,703986,106546942,True,False,English
4,Tfue,3671000070,123660,285644,29602,8938903,2068424,78998587,True,False,English


In [26]:
mature_streamers = df[df["Mature"] == True]
top_mature_streamers = mature_streamers.nlargest(200, "Followers")

In [27]:
top_mature_streamers.head()

,Channel,Watch time(Minutes),Stream time(minutes),Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language
10,TimTheTatman,2834436990,108780,142067,25664,5265659,1244341,50119786,True,True,English
15,MontanaBlack88,2408460990,67740,181600,33514,2911316,1101093,37189666,True,True,German
16,sodapoppin,2329440420,115305,107833,19659,2786162,236169,39334821,True,True,English
74,Symfuhny,1076179485,137400,45671,7327,2355063,704327,18756705,True,True,English
167,wtcN,582125625,77385,73861,7438,1852272,566210,25333548,True,True,Turkish


#### Selection of Criteria for STREAMER Ranking

For ranking TV shows logically, the **7 best criteria** to use for decision-making are:

1. **Followers gained** – Represents the popularity growth of the show.  
2. **Views gained** – Indicates how much attention the show received.  
3. **Peak viewers** – Shows the maximum number of viewers at once (indicates viral moments).  
4. **Average viewers** – Represents consistent engagement over time.  
5. **Watch time (Minutes)** – Measures total audience consumption (engagement metric).  
6. **Stream time (Minutes)** – Helps understand how long the content was available.  
7. **Partnered** (Boolean: 1 for Yes, 0 for No) – Partnership status can indicate official recognition and influence.  


In [28]:
criteria = [
    "Followers gained", "Views gained", "Peak viewers",
    "Average viewers", "Watch time(Minutes)", "Stream time(minutes)", "Partnered"
]

In [29]:
df_final = top_mature_streamers[["Channel"] + criteria]

In [30]:
df_final

,Channel,Followers gained,Views gained,Peak viewers,Average viewers,Watch time(Minutes),Stream time(minutes),Partnered
10,TimTheTatman,1244341,50119786,142067,25664,2834436990,108780,True
15,MontanaBlack88,1101093,37189666,181600,33514,2408460990,67740,True
16,sodapoppin,236169,39334821,107833,19659,2329440420,115305,True
74,Symfuhny,704327,18756705,45671,7327,1076179485,137400,True
167,wtcN,566210,25333548,73861,7438,582125625,77385,True
2,Gaules,1023779,102611607,387315,10976,5644590915,515280,True
218,Aydan,425036,11009240,51268,3507,495563580,140595,True
77,Rainbow6,428942,16786627,135471,11535,1031011170,82380,True
301,Trick2g,115863,8688676,28917,2650,371417130,132945,True
779,Dyrus,111,3724340,11338,1123,158999070,138300,True


### II- AHP WEIGHTS

In [31]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns (if needed)
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping

In [32]:
# Given AHP Pairwise Comparison Matrix
criteria_matrix = np.array([
    [1.00, 1.00, 0.50, 0.33, 0.67, 3.33, 1.00],
    [1.00, 1.00, 0.67, 0.50, 0.67, 2.00, 1.00],
    [2.00, 1.50, 1.00, 1.00, 3.00, 3.00, 1.00],
    [3.00, 2.00, 1.00, 1.00, 3.00, 3.00, 1.00],
    [1.50, 0.33, 0.33, 0.33, 1.00, 2.50, 0.50],
    [0.30, 0.50, 0.33, 0.33, 0.40, 1.00, 0.50],
    [1.00, 1.00, 1.00, 1.00, 2.00, 2.00, 1.00]
])

# Random Consistency Index (RI) values for different matrix sizes
RI = {
    1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32,
    8: 1.41, 9: 1.45, 10: 1.49, 11: 1.51, 12: 1.48, 13: 1.56, 14: 1.57, 15: 1.59
}

# Step 1: Normalize the pairwise comparison matrix
column_sums = criteria_matrix.sum(axis=0)
normalized_matrix = criteria_matrix / column_sums

# Step 2: Compute the AHP weights (average of normalized values per row)
criteria_weights = normalized_matrix.mean(axis=1)

# Step 3: Compute the AW Matrix (Multiply original matrix by weight vector)
AW = criteria_matrix.dot(criteria_weights)

# Step 4: Compute Lambda Max (λ_max) as the average of AW / criteria_weights
lambda_max = np.mean(AW / criteria_weights)

# Step 5: Compute Consistency Index (CI)
n = criteria_matrix.shape[0]  # Number of criteria
CI = (lambda_max - n) / (n - 1)

# Step 6: Compute Consistency Ratio (CR)
RI_n = RI.get(n, 1.6)  # Get RI for 'n' criteria, default to 1.6 if not found
CR = CI / RI_n

# Display results
print("\nComputed AHP Criteria Weights:")
print(np.round(criteria_weights, 3))

print(f"\nLambda Max (λ_max): {lambda_max:.4f}")
print(f"Consistency Index (CI): {CI:.4f}")
print(f"Consistency Ratio (CR): {CR:.4f}")


Computed AHP Criteria Weights:
[0.12  0.119 0.209 0.233 0.095 0.06  0.163]

Lambda Max (λ_max): 7.1126
Consistency Index (CI): 0.0188
Consistency Ratio (CR): 0.0142


In [33]:
# Given AHP Weights
ahp_weights = np.array([0.12, 0.119, 0.209, 0.233, 0.095, 0.06, 0.163])

#### 1- WSM, WPM and WASPAS 

In [34]:
# ------------------------------------------------------------------------------
# Function Definitions
# ------------------------------------------------------------------------------
def generate_criteria_type_matrix(m):
    """Generate criteria type matrix (1 = maximization criteria)."""
    return np.ones((1, m))  # All criteria are maximization

def normalize_matrix_max(mat, criteria_types):
    """Normalize decision matrix using max normalization."""
    normalized_matrix = np.zeros_like(mat, dtype=float)
    for j in range(mat.shape[1]):
        if criteria_types[0, j] == 1:  # Maximization
            normalized_matrix[:, j] = mat[:, j] / np.max(mat[:, j])
    return normalized_matrix

def compute_wsm_score(normalized_matrix, weights):
    """Compute Weighted Sum Model (WSM) scores."""
    return np.sum(normalized_matrix * weights, axis=1)

def compute_wpm_score(normalized_matrix, weights):
    """Compute Weighted Product Model (WPM) scores."""
    return np.prod(normalized_matrix ** weights, axis=1)

def compute_wasspass_score(wsm_scores, wpm_scores, lambda_value=0.5):
    """Compute WASSPASS scores (combining WSM and WPM)."""
    return lambda_value * wsm_scores + (1 - lambda_value) * wpm_scores

# ------------------------------------------------------------------------------
# Data Processing and Ranking
# ------------------------------------------------------------------------------

# Extract decision matrix
decision_matrix = df_final[criteria].values

# Normalize the matrix using max normalization
m = decision_matrix.shape[1]
criteria_types = generate_criteria_type_matrix(m)
normalized_matrix = normalize_matrix_max(decision_matrix, criteria_types)

# Compute scores using AHP weights
wsm_scores = compute_wsm_score(normalized_matrix, ahp_weights)
wpm_scores = compute_wpm_score(normalized_matrix, ahp_weights)
wasspass_scores = compute_wasspass_score(wsm_scores, wpm_scores, lambda_value=0.5)

# Append scores to DataFrame
df_final["WSM Score"] = wsm_scores
df_final["WPM Score"] = wpm_scores
df_final["WASSPASS Score"] = wasspass_scores

# Rank alternatives for each method separately
df_wsm_ranking = df_final.sort_values(by="WSM Score", ascending=False)
df_wpm_ranking = df_final.sort_values(by="WPM Score", ascending=False)
df_wasspass_ranking = df_final.sort_values(by="WASSPASS Score", ascending=False)

# ------------------------------------------------------------------------------
# Display Results in Separate Tables
# ------------------------------------------------------------------------------
print("\nRanking Based on WSM Score:")
print(df_wsm_ranking[["Channel", "WSM Score"]])

print("\nRanking Based on WPM Score:")
print(df_wpm_ranking[["Channel", "WPM Score"]])

print("\nRanking Based on WASSPASS Score:")
print(df_wasspass_ranking[["Channel", "WASSPASS Score"]])


Ranking Based on WSM Score:
                       Channel  WSM Score
2                       Gaules   0.821038
15              MontanaBlack88   0.691731
24               TheRealKnossi   0.660400
10                TimTheTatman   0.656581
16                  sodapoppin   0.478887
84                     ZeratoR   0.464212
212                 AustinShow   0.435001
45                    Sardoche   0.411978
927  Call of Duty (callofduty)   0.406906
171                Reborn_Live   0.406643
77                    Rainbow6   0.404075
431                   JokerdTV   0.393887
74                    Symfuhny   0.362371
167                       wtcN   0.357359
119                       aceu   0.339449
176            KendineMuzisyen   0.336774
489                    Arteezy   0.336042
568             ElcanaldeJoaco   0.333971
988           CallMeCarsonLIVE   0.331715
55                       Gorgc   0.328769
80               Trainwreckstv   0.324803
71                      forsen   0.320130
32   

C:\Users\amine\AppData\Local\Temp\ipykernel_20636\2322289756.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["WSM Score"] = wsm_scores
C:\Users\amine\AppData\Local\Temp\ipykernel_20636\2322289756.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["WPM Score"] = wpm_scores


#### 2- TOPSIS

In [35]:
# Define max/min criteria (modifiable)
criteria_types = ['max', 'max', 'max', 'max', 'max', 'max', 'max']

# Given AHP weights
ahp_weights = np.array([0.12, 0.119, 0.209, 0.233, 0.095, 0.06, 0.163])

# Convert Partnered (True/False) to numeric
df_final["Partnered"] = df_final["Partnered"].astype(int)

# Extract decision matrix (X) and Channel names
X = df_final[criteria].values
channels = df_final["Channel"].values

# Step 1: Normalize the decision matrix
norms = np.linalg.norm(X, axis=0)
X_norm = X / norms

# Step 2: Adjust for min/max criteria
for i, ctype in enumerate(criteria_types):
    if ctype == 'min':  
        X_norm[:, i] = 1 - X_norm[:, i]  # Flip values for minimization

# Step 3: Multiply by AHP weights
X_weighted = X_norm * ahp_weights

# Step 4: Compute ideal and anti-ideal solutions
ideal_solution = np.max(X_weighted, axis=0)   # Best values
anti_ideal_solution = np.min(X_weighted, axis=0)  # Worst values

# Step 5: Compute distances to ideal and anti-ideal solutions
dist_ideal = np.sqrt(np.sum((X_weighted - ideal_solution) ** 2, axis=1))
dist_anti_ideal = np.sqrt(np.sum((X_weighted - anti_ideal_solution) ** 2, axis=1))

# Step 6: Compute TOPSIS scores
topsis_score = dist_anti_ideal / (dist_ideal + dist_anti_ideal)

# Step 7: Rank alternatives
df_final["TOPSIS_Score"] = topsis_score
df_final["Rank"] = df_final["TOPSIS_Score"].rank(ascending=False)

# Step 8: Sort by rank
df_final_sorted = df_final.sort_values(by="Rank")

# Display final ranking
print("\nTOPSIS Results (AHP WEIGHTS):")
print(df_final_sorted[["Channel", "TOPSIS_Score", "Rank"]])


TOPSIS Results (AHP WEIGHTS):
                       Channel  TOPSIS_Score   Rank
2                       Gaules      0.691764    1.0
15              MontanaBlack88      0.595617    2.0
24               TheRealKnossi      0.589476    3.0
10                TimTheTatman      0.548393    4.0
84                     ZeratoR      0.424820    5.0
16                  sodapoppin      0.403691    6.0
431                   JokerdTV      0.393331    7.0
927  Call of Duty (callofduty)      0.374119    8.0
212                 AustinShow      0.373712    9.0
171                Reborn_Live      0.304913   10.0
45                    Sardoche      0.304650   11.0
77                    Rainbow6      0.300615   12.0
489                    Arteezy      0.286545   13.0
167                       wtcN      0.224518   14.0
42                    ROSHTEIN      0.223916   15.0
168                      Anton      0.215275   16.0
74                    Symfuhny      0.214888   17.0
32                    MOONMOON   

#### 3- PROMETHEE I and PROMETHEE II

In [36]:
# Extract relevant data
df_final = top_mature_streamers[["Channel"] + criteria]
matrix = df_final[criteria].values  # Convert to NumPy array
n, m = matrix.shape

# Define AHP weights
weights = np.array([0.12, 0.119, 0.209, 0.233, 0.095, 0.06, 0.163])
weights /= np.sum(weights)  # Normalize weights

# Define criteria types (1 = maximization, all are beneficial)
criteria_types = np.ones(m, dtype=int)

# PROMETHEE Functions
def normalize_promethee(matrix):
    """Normalize the decision matrix for PROMETHEE."""
    n, m = matrix.shape
    normalized_matrix = np.zeros((n, m), dtype=float)
    
    for j in range(m):
        col_min, col_max = np.min(matrix[:, j]), np.max(matrix[:, j])
        if col_max == col_min:  
            normalized_matrix[:, j] = 0.5  # Neutral value instead of 1
        else:
            normalized_matrix[:, j] = (matrix[:, j] - col_min) / (col_max - col_min)
    
    return normalized_matrix

def compute_promethee(matrix, weights):
    """Compute PROMETHEE I (partial ranking) and PROMETHEE II (complete ranking)."""
    n, m = matrix.shape
    normalized_matrix = normalize_promethee(matrix)

    # Compute pairwise preference differences
    pairwise_differences = np.zeros((n, n, m))
    for i in range(n):
        for j in range(n):
            pairwise_differences[i, j, :] = normalized_matrix[i, :] - normalized_matrix[j, :]

    # Compute preference function Pj(a,b)
    preference_matrix = np.maximum(pairwise_differences, 0)

    # Weighted preference matrix
    weighted_preference_matrix = np.multiply(preference_matrix, weights)

    # Priority matrix
    priority_matrix = np.sum(weighted_preference_matrix, axis=2)

    # Compute Phi+ (positive flow) and Phi- (negative flow)
    phi_plus = np.sum(priority_matrix, axis=1) / (n - 1)
    phi_minus = np.sum(priority_matrix, axis=0) / (n - 1)
    phi_net = phi_plus - phi_minus

    # -----------------------------
    # PROMETHEE I (Partial Ranking)
    # -----------------------------
    ranking_df = pd.DataFrame({
        "Channel": df_final["Channel"],
        "Phi+": phi_plus,
        "Phi-": phi_minus
    }).sort_values(by="Phi+", ascending=False).reset_index(drop=True)

    ranking_groups = {}
    for i in range(n):
        alternative = ranking_df.iloc[i]["Channel"]
        phi_p, phi_m = ranking_df.iloc[i]["Phi+"], ranking_df.iloc[i]["Phi-"]

        # Define relationships
        rank_label = f"Phi+={phi_p:.3f}, Phi-={phi_m:.3f}"
        if rank_label not in ranking_groups:
            ranking_groups[rank_label] = []
        ranking_groups[rank_label].append(alternative)

    # -----------------------------
    # PROMETHEE II (Complete Ranking)
    # -----------------------------
    ranking_df["Phi_Net"] = phi_net
    ranking_df = ranking_df.sort_values(by="Phi_Net", ascending=False).reset_index(drop=True)

    return normalized_matrix, priority_matrix, phi_plus, phi_minus, phi_net, ranking_groups, ranking_df

# Compute PROMETHEE results
normalized_matrix, priority_matrix, phi_plus, phi_minus, phi_net, ranking_groups, ranking_df = compute_promethee(matrix, weights)

# Display results
print("\nPROMETHEE I Ranking: (AHP WEIGHTS)")
for rank, alternatives in ranking_groups.items():
    print(f"{rank} -> {', '.join(alternatives)}")

# Show ranking dataframe
print("\nPROMETHEE II Ranking: (AHP WEIGHTS)")
print(ranking_df)  # Display top-ranked streamers



PROMETHEE I Ranking: (AHP WEIGHTS)
Phi+=0.584, Phi-=0.004 -> Gaules
Phi+=0.458, Phi-=0.009 -> MontanaBlack88
Phi+=0.427, Phi-=0.010 -> TheRealKnossi
Phi+=0.419, Phi-=0.006 -> TimTheTatman
Phi+=0.246, Phi-=0.013 -> sodapoppin
Phi+=0.234, Phi-=0.016 -> ZeratoR
Phi+=0.212, Phi-=0.024 -> AustinShow
Phi+=0.197, Phi-=0.037 -> Call of Duty (callofduty)
Phi+=0.182, Phi-=0.023 -> Reborn_Live
Phi+=0.177, Phi-=0.013 -> Sardoche
Phi+=0.177, Phi-=0.031 -> JokerdTV
Phi+=0.172, Phi-=0.016 -> Rainbow6
Phi+=0.130, Phi-=0.016 -> Symfuhny
Phi+=0.129, Phi-=0.020 -> wtcN
Phi+=0.127, Phi-=0.039 -> Arteezy
Phi+=0.120, Phi-=0.180 -> ROSHTEIN
Phi+=0.116, Phi-=0.034 -> CallMeCarsonLIVE
Phi+=0.116, Phi-=0.031 -> ElcanaldeJoaco
Phi+=0.112, Phi-=0.021 -> aceu
Phi+=0.111, Phi-=0.023 -> KendineMuzisyen
Phi+=0.102, Phi-=0.021 -> Gorgc
Phi+=0.097, Phi-=0.026 -> MOONMOON
Phi+=0.095, Phi-=0.027 -> scoped
Phi+=0.095, Phi-=0.023 -> forsen
Phi+=0.095, Phi-=0.019 -> Trainwreckstv
Phi+=0.094, Phi-=0.025 -> INSCOPE21TV
Phi+=

### III- ENTROPY WEIGHTS

In [37]:
# Function to normalize the matrix column-wise so that each column sums to 1.
def normalize_matrix_columnwise(mat):
    col_sums = np.sum(mat, axis=0)
    return mat / col_sums

# Function to compute the entropy for each criterion (column)
def compute_entropy(matrix):
    n, m = matrix.shape  # n: number of alternatives, m: number of criteria
    k = 1 / np.log(float(n))  # Ensure n is treated as a float
    epsilon = 1e-10  # Small constant to avoid log(0)
    
    # Ensure matrix is of type float64
    matrix = np.array(matrix, dtype=np.float64)
    
    # Compute entropy formula
    pij_ln_pij = np.where(matrix > 0, matrix * np.log1p(matrix - 1 + epsilon), 0)
    column_sums = np.sum(pij_ln_pij, axis=0)
    entropy_values = -k * column_sums
    return entropy_values

# --- Assuming df_final and criteria are already defined ---
decision_matrix = df_final[criteria].values

# Ensure decision_matrix is a float64 array
decision_matrix = np.array(decision_matrix, dtype=np.float64)

# Normalize the decision matrix column-wise
normalized_matrix_entropy = normalize_matrix_columnwise(decision_matrix)

# Compute entropy values for each criterion
entropy_values = compute_entropy(normalized_matrix_entropy)

# Compute diversity (d_j = 1 - E_j)
diversity = 1 - entropy_values

# Compute entropy weights (w_j = d_j / sum(d_j))
entropy_weights = diversity / np.sum(diversity)

# Display the entropy weights
print("Entropy Weights for each criterion:")
print(entropy_weights)

Entropy Weights for each criterion:
[0.21041816 0.14298389 0.25862418 0.16473849 0.16506985 0.05258346
 0.00558197]


In [38]:
entropy_weights = np.array([0.21041816, 0.14298389, 0.25862418, 0.16473849, 
                            0.16506985, 0.05258346, 0.00558197])

#### 1- WSM, WPM and WASPAS 

In [39]:
# ------------------------------------------------------------------------------
# Function Definitions
# ------------------------------------------------------------------------------
def generate_criteria_type_matrix(m):
    """Generate criteria type matrix (1 = maximization criteria)."""
    return np.ones((1, m))  # All criteria are maximization

def normalize_matrix_max(mat, criteria_types):
    """Normalize decision matrix using max normalization."""
    normalized_matrix = np.zeros_like(mat, dtype=float)
    for j in range(mat.shape[1]):
        if criteria_types[0, j] == 1:  # Maximization
            normalized_matrix[:, j] = mat[:, j] / np.max(mat[:, j])
    return normalized_matrix

def compute_wsm_score(normalized_matrix, weights):
    """Compute Weighted Sum Model (WSM) scores."""
    return np.sum(normalized_matrix * weights, axis=1)

def compute_wpm_score(normalized_matrix, weights):
    """Compute Weighted Product Model (WPM) scores."""
    return np.prod(normalized_matrix ** weights, axis=1)

def compute_wasspass_score(wsm_scores, wpm_scores, lambda_value=0.5):
    """Compute WASSPASS scores (combining WSM and WPM)."""
    return lambda_value * wsm_scores + (1 - lambda_value) * wpm_scores

# ------------------------------------------------------------------------------
# Data Processing and Ranking
# ------------------------------------------------------------------------------

# Extract decision matrix
decision_matrix = df_final[criteria].values

# Normalize the matrix using max normalization
m = decision_matrix.shape[1]
criteria_types = generate_criteria_type_matrix(m)
normalized_matrix = normalize_matrix_max(decision_matrix, criteria_types)

# Compute scores using Entropy Weights
wsm_scores = compute_wsm_score(normalized_matrix, entropy_weights)
wpm_scores = compute_wpm_score(normalized_matrix, entropy_weights)
wasspass_scores = compute_wasspass_score(wsm_scores, wpm_scores, lambda_value=0.5)

# Append scores to DataFrame
df_final["WSM Score"] = wsm_scores
df_final["WPM Score"] = wpm_scores
df_final["WASSPASS Score"] = wasspass_scores

# Rank alternatives for each method separately
df_wsm_ranking = df_final.sort_values(by="WSM Score", ascending=False)
df_wpm_ranking = df_final.sort_values(by="WPM Score", ascending=False)
df_wasspass_ranking = df_final.sort_values(by="WASSPASS Score", ascending=False)

# ------------------------------------------------------------------------------
# Display Results in Separate Tables
# ------------------------------------------------------------------------------
print("\nRanking Based on WSM Score (Entropy Weights):")
print(df_wsm_ranking[["Channel", "WSM Score"]])

print("\nRanking Based on WPM Score (Entropy Weights):")
print(df_wpm_ranking[["Channel", "WPM Score"]])

print("\nRanking Based on WASSPASS Score (Entropy Weights):")
print(df_wasspass_ranking[["Channel", "WASSPASS Score"]])



Ranking Based on WSM Score (Entropy Weights):
                       Channel  WSM Score
2                       Gaules   0.851917
15              MontanaBlack88   0.606944
10                TimTheTatman   0.600845
24               TheRealKnossi   0.596077
84                     ZeratoR   0.355130
16                  sodapoppin   0.348856
45                    Sardoche   0.305573
431                   JokerdTV   0.288571
77                    Rainbow6   0.287224
171                Reborn_Live   0.279654
212                 AustinShow   0.272707
74                    Symfuhny   0.262825
167                       wtcN   0.247431
119                       aceu   0.239227
927  Call of Duty (callofduty)   0.237256
568             ElcanaldeJoaco   0.223616
988           CallMeCarsonLIVE   0.220812
339                     scoped   0.213497
176            KendineMuzisyen   0.211066
42                    ROSHTEIN   0.208175
80               Trainwreckstv   0.200248
231                INSCOPE21T

C:\Users\amine\AppData\Local\Temp\ipykernel_20636\2698668945.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["WSM Score"] = wsm_scores
C:\Users\amine\AppData\Local\Temp\ipykernel_20636\2698668945.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["WPM Score"] = wpm_scores


#### 2- TOPSIS

In [40]:
# Define max/min criteria 
criteria_types = ['max', 'max', 'max', 'max', 'max', 'max', 'max']

# Given Entropy Weights
entropy_weights = np.array([0.21041816, 0.14298389, 0.25862418, 0.16473849, 
                            0.16506985, 0.05258346, 0.00558197])

# Convert Partnered (True/False) to numeric
df_final["Partnered"] = df_final["Partnered"].astype(int)

# Extract decision matrix (X) and Channel names
X = df_final[criteria].values
channels = df_final["Channel"].values

# Step 1: Normalize the decision matrix
norms = np.linalg.norm(X, axis=0)
X_norm = X / norms

# Step 2: Adjust for min/max criteria
for i, ctype in enumerate(criteria_types):
    if ctype == 'min':  
        X_norm[:, i] = 1 - X_norm[:, i]  # Flip values for minimization

# Step 3: Multiply by Entropy Weights
X_weighted = X_norm * entropy_weights

# Step 4: Compute ideal and anti-ideal solutions
ideal_solution = np.max(X_weighted, axis=0)   # Best values
anti_ideal_solution = np.min(X_weighted, axis=0)  # Worst values

# Step 5: Compute distances to ideal and anti-ideal solutions
dist_ideal = np.sqrt(np.sum((X_weighted - ideal_solution) ** 2, axis=1))
dist_anti_ideal = np.sqrt(np.sum((X_weighted - anti_ideal_solution) ** 2, axis=1))

# Step 6: Compute TOPSIS scores
topsis_score = dist_anti_ideal / (dist_ideal + dist_anti_ideal)

# Step 7: Rank alternatives
df_final["TOPSIS_Score"] = topsis_score
df_final["Rank"] = df_final["TOPSIS_Score"].rank(ascending=False)

# Step 8: Sort by rank
df_final_sorted = df_final.sort_values(by="Rank")

# Display all rows
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns (if needed)
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping

# Print full results
print("\nTOPSIS Results (Ranked) using Entropy Weights:")
print(df_final_sorted[["Channel", "TOPSIS_Score", "Rank"]])



TOPSIS Results (Ranked) using Entropy Weights:
                       Channel  TOPSIS_Score   Rank
2                       Gaules      0.803342    1.0
24               TheRealKnossi      0.554518    2.0
15              MontanaBlack88      0.534056    3.0
10                TimTheTatman      0.527581    4.0
84                     ZeratoR      0.409998    5.0
431                   JokerdTV      0.400846    6.0
16                  sodapoppin      0.349302    7.0
45                    Sardoche      0.304340    8.0
927  Call of Duty (callofduty)      0.289279    9.0
77                    Rainbow6      0.278356   10.0
212                 AustinShow      0.261847   11.0
171                Reborn_Live      0.252579   12.0
74                    Symfuhny      0.234222   13.0
167                       wtcN      0.225226   14.0
119                       aceu      0.221134   15.0
568             ElcanaldeJoaco      0.217668   16.0
988           CallMeCarsonLIVE      0.217514   17.0
168             

#### 3- PROMETHEE I and PROMETHEE II

In [41]:
# Extract relevant data
df_final = top_mature_streamers[["Channel"] + criteria]
matrix = df_final[criteria].values  # Convert to NumPy array
n, m = matrix.shape

# Define AHP weights
weights = np.array([0.21041816, 0.14298389, 0.25862418, 0.16473849, 0.16506985, 0.05258346, 0.00558197])
weights /= np.sum(weights)  # Normalize weights

# Define criteria types (1 = maximization, adjust if needed)
criteria_types = np.ones(m, dtype=int)  # Assuming all are beneficial

# PROMETHEE Functions
def normalize_promethee(matrix, criteria_types):
    """Normalize the decision matrix for PROMETHEE."""
    n, m = matrix.shape
    normalized_matrix = np.zeros((n, m), dtype=float)
    
    for j in range(m):
        col_min, col_max = np.min(matrix[:, j]), np.max(matrix[:, j])
        if col_max == col_min:  # Avoid division by zero
            normalized_matrix[:, j] = 1
        else:
            normalized_matrix[:, j] = (matrix[:, j] - col_min) / (col_max - col_min)
    
    return normalized_matrix

def compute_promethee(matrix, criteria_types, weights):
    """Compute PROMETHEE I (partial ranking) and PROMETHEE II (complete ranking)."""
    n, m = matrix.shape
    normalized_matrix = normalize_promethee(matrix, criteria_types)

    # Compute pairwise preference differences
    pairwise_differences = np.zeros((n, n, m))
    for i in range(n):
        for j in range(n):
            pairwise_differences[i, j, :] = normalized_matrix[i, :] - normalized_matrix[j, :]

    # Compute preference function Pj(a,b)
    preference_matrix = np.maximum(pairwise_differences, 0)

    # Weighted preference matrix
    weighted_preference_matrix = np.multiply(preference_matrix, weights)

    # Priority matrix
    priority_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            priority_matrix[i, j] = np.sum(weighted_preference_matrix[i, j, :])

    # Compute Phi+ (positive flow) and Phi- (negative flow)
    phi_plus = np.sum(priority_matrix, axis=1) / (n - 1)
    phi_minus = np.sum(priority_matrix, axis=0) / (n - 1)
    phi_net = phi_plus - phi_minus

    # -----------------------------
    # PROMETHEE I (Partial Ranking)
    # -----------------------------
    ranking_df = pd.DataFrame({
        "Channel": df_final["Channel"],
        "Phi+": phi_plus,
        "Phi-": phi_minus
    }).sort_values(by="Phi+", ascending=False).reset_index(drop=True)

    # Group alternatives based on Phi+ and Phi-
    ranking_groups = {}
    for i in range(n):
        alternative = ranking_df.iloc[i]["Channel"]
        phi_p, phi_m = ranking_df.iloc[i]["Phi+"], ranking_df.iloc[i]["Phi-"]

        # Define relationships
        rank_label = f"Phi+={phi_p:.3f}, Phi-={phi_m:.3f}"
        if rank_label not in ranking_groups:
            ranking_groups[rank_label] = []
        ranking_groups[rank_label].append(alternative)

    # -----------------------------
    # PROMETHEE II (Complete Ranking)
    # -----------------------------
    ranking_df["Phi_Net"] = phi_net
    ranking_df = ranking_df.sort_values(by="Phi_Net", ascending=False).reset_index(drop=True)

    return normalized_matrix, priority_matrix, phi_plus, phi_minus, phi_net, ranking_groups, ranking_df

# Compute PROMETHEE results
normalized_matrix, priority_matrix, phi_plus, phi_minus, phi_net, ranking_groups, ranking_df = compute_promethee(matrix, criteria_types, weights)

# Display results
print("\nPROMETHEE I Ranking: (AHP WEIGHTS)")
for rank, alternatives in ranking_groups.items():
    print(f"{rank} -> {', '.join(alternatives)}")

# Show ranking dataframe
print("\nPROMETHEE II Ranking: (AHP WEIGHTS)")
print(ranking_df)  # Display top-ranked streamers



PROMETHEE I Ranking: (AHP WEIGHTS)
Phi+=0.758, Phi-=0.003 -> Gaules
Phi+=0.515, Phi-=0.009 -> MontanaBlack88
Phi+=0.506, Phi-=0.007 -> TimTheTatman
Phi+=0.504, Phi-=0.010 -> TheRealKnossi
Phi+=0.270, Phi-=0.019 -> ZeratoR
Phi+=0.262, Phi-=0.017 -> sodapoppin
Phi+=0.217, Phi-=0.035 -> JokerdTV
Phi+=0.215, Phi-=0.015 -> Sardoche
Phi+=0.199, Phi-=0.018 -> Rainbow6
Phi+=0.198, Phi-=0.025 -> Reborn_Live
Phi+=0.195, Phi-=0.027 -> AustinShow
Phi+=0.180, Phi-=0.049 -> Call of Duty (callofduty)
Phi+=0.174, Phi-=0.018 -> Symfuhny
Phi+=0.162, Phi-=0.021 -> wtcN
Phi+=0.155, Phi-=0.022 -> aceu
Phi+=0.151, Phi-=0.037 -> CallMeCarsonLIVE
Phi+=0.150, Phi-=0.034 -> ElcanaldeJoaco
Phi+=0.135, Phi-=0.029 -> scoped
Phi+=0.131, Phi-=0.029 -> ROSHTEIN
Phi+=0.130, Phi-=0.025 -> KendineMuzisyen
Phi+=0.119, Phi-=0.038 -> Pimpeano
Phi+=0.115, Phi-=0.022 -> Trainwreckstv
Phi+=0.115, Phi-=0.027 -> INSCOPE21TV
Phi+=0.110, Phi-=0.027 -> Gorgc
Phi+=0.103, Phi-=0.025 -> TeePee
Phi+=0.102, Phi-=0.034 -> Anton
Phi+=0.

### IV- Computational Time Comparaison

Below are the execution times (in seconds) for different Multi-Criteria Decision-Making (MCDM) methods across varying problem sizes.  
The analysis compares **WSM**, **WPM**, **WASPAS**, **TOPSIS**, and **PROMETHEE** for up to **10,000 alternatives** and **500 criteria** for further study purposes.

In [42]:
# --- Core Functions ---
def generate_random_matrix(n, m):
    """Generates a random decision matrix of shape (n, m)."""
    return np.random.rand(n, m)

def compute_wsm_score(matrix, weights):
    """Computes WSM scores."""
    return np.sum(matrix * weights, axis=1)

def compute_wpm_score(matrix, weights):
    """Computes WPM scores."""
    return np.prod(matrix ** weights, axis=1)

def compute_wasspass_score(wsm_scores, wpm_scores, lambda_value=0.5):
    """Computes WASPAS scores."""
    return lambda_value * wsm_scores + (1 - lambda_value) * wpm_scores

def compute_topsis(matrix, weights):
    """Computes TOPSIS scores."""
    X_norm = matrix / np.linalg.norm(matrix, axis=0)
    X_weighted = X_norm * weights
    ideal_solution = np.max(X_weighted, axis=0)
    anti_ideal_solution = np.min(X_weighted, axis=0)
    dist_ideal = np.sqrt(np.sum((X_weighted - ideal_solution) ** 2, axis=1))
    dist_anti_ideal = np.sqrt(np.sum((X_weighted - anti_ideal_solution) ** 2, axis=1))
    return dist_anti_ideal / (dist_ideal + dist_anti_ideal)

def compute_promethee(matrix, weights):
    """Computes PROMETHEE scores (simplified version)."""
    n, m = matrix.shape
    normalized_matrix = (matrix - np.min(matrix, axis=0)) / (np.max(matrix, axis=0) - np.min(matrix, axis=0))
    pairwise_diff = normalized_matrix[:, np.newaxis, :] - normalized_matrix[np.newaxis, :, :]
    preference_matrix = np.maximum(pairwise_diff, 0)
    weighted_preference = preference_matrix * weights
    priority_matrix = np.sum(weighted_preference, axis=2)
    phi_plus = np.mean(priority_matrix, axis=1)
    phi_minus = np.mean(priority_matrix, axis=0)
    return phi_plus - phi_minus  # Net flow scores

# --- Benchmark Settings ---
problem_sizes = [
    (50, 10), (100, 10), 
    (200, 20), (500, 20), 
    (1000, 50), (2000, 100), 
    (5000, 200), (10000, 500)
]

# --- Performance Testing ---
results = []
for n, m in problem_sizes:
    print(f"Processing {n} alternatives and {m} criteria...")
    matrix = generate_random_matrix(n, m)
    weights = np.random.rand(m)
    weights /= np.sum(weights)  # Normalize weights

    # Time WSM
    start = time.time()
    _ = compute_wsm_score(matrix, weights)
    wsm_time = time.time() - start

    # Time WPM
    start = time.time()
    _ = compute_wpm_score(matrix, weights)
    wpm_time = time.time() - start

    # Time WASPAS
    start = time.time()
    wsm = compute_wsm_score(matrix, weights)
    wpm = compute_wpm_score(matrix, weights)
    _ = compute_wasspass_score(wsm, wpm)
    wasspass_time = time.time() - start

    # Time TOPSIS
    start = time.time()
    _ = compute_topsis(matrix, weights)
    topsis_time = time.time() - start

    # Time PROMETHEE (skip for large n)
    promethee_time = np.nan
    if n <= 2000:
        start = time.time()
        _ = compute_promethee(matrix, weights)
        promethee_time = time.time() - start

    results.append([n, m, wsm_time, wpm_time, wasspass_time, topsis_time, promethee_time])

# --- Results ---
df_results = pd.DataFrame(results, columns=[
    "Alternatives", "Criteria", 
    "WSM Time", "WPM Time", 
    "WASSPAS Time", "TOPSIS Time", 
    "PROMETHEE Time"
])
print(df_results)

Processing 50 alternatives and 10 criteria...
Processing 100 alternatives and 10 criteria...
Processing 200 alternatives and 20 criteria...
Processing 500 alternatives and 20 criteria...
Processing 1000 alternatives and 50 criteria...
Processing 2000 alternatives and 100 criteria...
Processing 5000 alternatives and 200 criteria...
Processing 10000 alternatives and 500 criteria...
   Alternatives  Criteria  WSM Time  WPM Time  WASSPAS Time  TOPSIS Time  PROMETHEE Time
0            50        10  0.000000  0.000000      0.000000     0.000000        0.000515
1           100        10  0.000529  0.000000      0.000000     0.000000        0.002111
2           200        20  0.000000  0.000000      0.000566     0.000000        0.013589
3           500        20  0.000000  0.000527      0.000000     0.000555        0.059898
4          1000        50  0.000520  0.001026      0.001030     0.000509        0.527261
5          2000       100  0.000520  0.005786      0.009140     0.006382        4.4